# Automação de Indicadores

### Objetivo: Treinar e criar um Projeto Completo que envolva a automatização de um processo feito no computador

### Descrição:

Imagine que você trabalha em uma grande rede de lojas de roupa com 25 lojas espalhadas por todo o Brasil.

Todo dia, pela manhã, a equipe de análise de dados calcula os chamados One Pages e envia para o gerente de cada loja o OnePage da sua loja, bem como todas as informações usadas no cálculo dos indicadores.

Um One Page é um resumo muito simples e direto ao ponto, usado pela equipe de gerência de loja para saber os principais indicadores de cada loja e permitir em 1 página (daí o nome OnePage) tanto a comparação entre diferentes lojas, quanto quais indicadores aquela loja conseguiu cumprir naquele dia ou não.

Exemplo de OnePage:

![title](onepage.png)

O seu papel, como Analista de Dados, é conseguir criar um processo da forma mais automática possível para calcular o OnePage de cada loja e enviar um email para o gerente de cada loja com o seu OnePage no corpo do e-mail e também o arquivo completo com os dados da sua respectiva loja em anexo.

Ex: O e-mail a ser enviado para o Gerente da Loja A deve ser como exemplo

![exemplo_email](Exemplo.JPG)

### Arquivos e Informações Importantes

- Arquivo Emails.xlsx com o nome, a loja e o e-mail de cada gerente. Obs: Sugiro substituir a coluna de e-mail de cada gerente por um e-mail seu, para você poder testar o resultado

- Arquivo Vendas.xlsx com as vendas de todas as lojas. Obs: Cada gerente só deve receber o OnePage e um arquivo em excel em anexo com as vendas da sua loja. As informações de outra loja não devem ser enviados ao gerente que não é daquela loja.

- Arquivo Lojas.csv com o nome de cada Loja

- Ao final, sua rotina deve enviar ainda um e-mail para a diretoria (informações também estão no arquivo Emails.xlsx) com 2 rankings das lojas em anexo, 1 ranking do dia e outro ranking anual. Além disso, no corpo do e-mail, deve ressaltar qual foi a melhor e a pior loja do dia e também a melhor e pior loja do ano. O ranking de uma loja é dado pelo faturamento da loja.

- As planilhas de cada loja devem ser salvas dentro da pasta da loja com a data da planilha, a fim de criar um histórico de backup

### Indicadores do OnePage

- Faturamento -> Meta Ano: 1.650.000 / Meta Dia: 1000
- Diversidade de Produtos (quantos produtos diferentes foram vendidos naquele período) -> Meta Ano: 120 / Meta Dia: 4
- Ticket Médio por Venda -> Meta Ano: 500 / Meta Dia: 500

Obs: Cada indicador deve ser calculado no dia e no ano. O indicador do dia deve ser o do último dia disponível na planilha de Vendas (a data mais recente)

Obs2: Dica para o caracter do sinal verde e vermelho: pegue o caracter desse site (https://fsymbols.com/keyboard/windows/alt-codes/list/) e formate com html

### Passo 1 - importar a base de dados

In [10]:
import pandas as pd
import os # manipula variáveis de ambiente do sistema (como pegar o e-mail e senha salvos no .env from dotenv import load_dotenv  #carrega automaticamente as variáveis definidas no arquivo .env
import smtplib #biblioteca padrão do Python para envio de e-mails via SMTP (protocolo do Gmail)
from email.message import EmailMessage  # facilita a criação e envio de e-mails com anexos e formatação
import pathlib
from dotenv import load_dotenv  # carrega automaticamente as variáveis definidas no arquivo .env

In [2]:
email = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')


In [3]:
display(email)
display(lojas)
display(vendas) 

,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,pythonimpressionador+helena@gmail.com
1,Shopping Midway Mall,Alice,pythonimpressionador+alice@gmail.com
2,Norte Shopping,Laura,pythonimpressionador+laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,pythonimpressionador+manuela@gmail.com
4,Shopping União de Osasco,Valentina,pythonimpressionador+valentina@gmail.com
5,Shopping Center Interlagos,Sophia,pythonimpressionador+sophia@gmail.com
6,Rio Mar Recife,Isabella,pythonimpressionador+isabella@gmail.com
7,Salvador Shopping,Heloisa,pythonimpressionador+heloisa@gmail.com
8,Rio Mar Shopping Fortaleza,Luiza,pythonimpressionador+luiza@gmail.com
9,Shopping Center Leste Aricanduva,Julia,pythonimpressionador+julia@gmail.com


,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco
5,6,Shopping Center Interlagos
6,7,Rio Mar Recife
7,8,Salvador Shopping
8,9,Rio Mar Shopping Fortaleza
9,10,Shopping Center Leste Aricanduva


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71
...,...,...,...,...,...,...,...
100994,69996,2019-12-26,17,Short Listrado,2,102,204
100995,69996,2019-12-26,17,Mochila,4,270,1080
100996,69996,2019-12-26,17,Pulseira Estampa,1,87,87
100997,69997,2019-12-26,11,Camisa Listrado,1,108,108


### Passo 2 - Definir Criar uma tabela para cada Loja e Definir o dia do Indicador

In [4]:
#incluir nome da loja em vendas
vendas = vendas.merge(lojas, on='ID Loja')
display(vendas)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,2,2019-01-02,3,Relógio,3,200,600,Norte Shopping
4,2,2019-01-02,3,Chinelo Liso,1,71,71,Norte Shopping
...,...,...,...,...,...,...,...,...
100994,69996,2019-12-26,17,Short Listrado,2,102,204,Center Shopping Uberlândia
100995,69996,2019-12-26,17,Mochila,4,270,1080,Center Shopping Uberlândia
100996,69996,2019-12-26,17,Pulseira Estampa,1,87,87,Center Shopping Uberlândia
100997,69997,2019-12-26,11,Camisa Listrado,1,108,108,Ribeirão Shopping


In [5]:
dicionario = {}

for loja in lojas['Loja']:
    dicionario[loja] = vendas.loc[vendas['Loja']==loja, : ]


In [6]:
dia_indicador = vendas['Data'].max()
print(dia_indicador)
print('{}/{}'.format(dia_indicador.day, dia_indicador.month))

2019-12-26 00:00:00
26/12


### Passo 3 - Salvar a planilha na pasta de backup

In [7]:
import pathlib

caminho_backup = pathlib.Path(r'Backup Arquivos Lojas') #define o caminho da pasta de backup

arquivos_pasta_backup = caminho_backup.iterdir() # iterdir() lista todos os arquivos da pasta dentro do caminho_backup

lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup] #lista os nomes dos arquivos na pasta de backup

for loja in dicionario:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja #cria o caminho da nova pasta
        nova_pasta.mkdir() #cria a nova pasta

    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja) #gera o nome do arquivo com o dia do indicador e o nome da loja
    print(nome_arquivo)
    local_arquivo = caminho_backup / loja / nome_arquivo  #Monta o caminho completo para onde o arquivo será salvo.

    dicionario[loja].to_excel(local_arquivo) #Salva o DataFrame correspondente à loja como um arquivo Excel naquele caminho.





  


12_26_Iguatemi Esplanada.xlsx
12_26_Shopping Midway Mall.xlsx
12_26_Norte Shopping.xlsx
12_26_Shopping Iguatemi Fortaleza.xlsx
12_26_Shopping União de Osasco.xlsx
12_26_Shopping Center Interlagos.xlsx
12_26_Rio Mar Recife.xlsx
12_26_Salvador Shopping.xlsx
12_26_Rio Mar Shopping Fortaleza.xlsx
12_26_Shopping Center Leste Aricanduva.xlsx
12_26_Ribeirão Shopping.xlsx
12_26_Shopping Morumbi.xlsx
12_26_Parque Dom Pedro Shopping.xlsx
12_26_Bourbon Shopping SP.xlsx
12_26_Palladium Shopping Curitiba.xlsx
12_26_Passei das Águas Shopping.xlsx
12_26_Center Shopping Uberlândia.xlsx
12_26_Shopping Recife.xlsx
12_26_Shopping Vila Velha.xlsx
12_26_Shopping SP Market.xlsx
12_26_Shopping Eldorado.xlsx
12_26_Shopping Ibirapuera.xlsx
12_26_Novo Shopping Ribeirão Preto.xlsx
12_26_Iguatemi Campinas.xlsx
12_26_Shopping Barra.xlsx


### Vamos definir as Metas

In [8]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000 # 1 milhão e seicentos e cinquenta 
meta_qtdprodutos_dia = 4
meta_qtdprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

### Passo 4 - Disparar o E-mail aos Gerentes

In [ ]:

for loja in dicionario:

    vendas_loja = dicionario[loja] # seleciona o DataFrame da loja específica
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :] # filtra as vendas do dia do indicador

    #faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum() # soma o faturamento do ano
    faturamento_dia = vendas_loja_dia['Valor Final'].sum() # soma o faturamento do dia do indicador

    #diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique()) # conta a quantidade de produtos únicos vendidos no ano 
    
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    
    #ticket medio ano 
    valor_venda = vendas_loja.groupby('Código Venda')['Valor Final'].sum()
    ticket_medio_ano = valor_venda.mean()

    #ticket medio dia 
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda')['Valor Final'].sum()
    ticket_medio_dia = valor_venda_dia.mean()


    # Carrega as variáveis do .env
    load_dotenv()

    # Recupera as credenciais de forma segura
    usuario = os.getenv('usuario')
    senha = os.getenv('senha')
    nome = email.loc[email['Loja'] == loja, 'Gerente'].values[0]
    destinatario = email.loc[email['Loja'] == loja, 'E-mail'].values[0]

    msg = EmailMessage()
    msg['From'] = usuario
    msg['To'] = destinatario
    msg['Subject'] = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}' # Assunto do e-mail
    msg.set_content('opa')  # Corpo do e-mail (modo texto)

    # Faturamento
    cor_fat_dia = 'green' if faturamento_dia >= meta_faturamento_dia else 'red'
    cor_fat_ano = 'green' if faturamento_ano >= meta_faturamento_ano else 'red'

    # Diversidade de Produtos
    cor_qtde_dia = 'green' if qtde_produtos_dia >= meta_qtdprodutos_dia else 'red'
    cor_qtde_ano = 'green' if qtde_produtos_ano >= meta_qtdprodutos_ano else 'red'

    # Ticket Médio
    cor_ticket_dia = 'green' if ticket_medio_dia >= meta_ticketmedio_dia else 'red'
    cor_ticket_ano = 'green' if ticket_medio_ano >= meta_ticketmedio_ano else 'red'


    msg.add_alternative (f'''

    <html>
    <body>
        <p>Bom dia, {nome}</p>

        <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da Loja <strong>{loja}</strong> foi:</p>

        <table border="1" cellpadding="6" cellspacing="0" style="border-collapse: collapse; margin: 10px 0;">
        <tr style="background-color: #f2f2f2;">
            <th>Indicador</th>
            <th>Valor Dia</th>
            <th>Meta Dia</th>
            <th>Cenário Dia</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: center">{faturamento_dia}</td>
            <td style="text-align: center">{meta_faturamento_dia}</td>
            <td style="text-align: center"><span style="font-size: 18px;">{'🟢' if cor_fat_dia == 'green' else '🔴'}</span></td>
        </tr>
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{qtde_produtos_dia}</td>
            <td style="text-align: center">{meta_qtdprodutos_dia}</td>
            <td style="text-align: center"><span style="font-size: 18px;">{'🟢' if cor_qtde_dia == 'green' else '🔴'}</span></td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">{ticket_medio_dia}</td>
            <td style="text-align: center">{meta_ticketmedio_dia}</td>
            <td style="text-align: center"><span style="font-size: 18px;">{'🟢' if cor_ticket_dia == 'green' else '🔴'}</span></td>
        </tr>
        </table>

        <table border="1" cellpadding="6" cellspacing="0" style="border-collapse: collapse; margin: 20px 0;">
        <tr style="background-color: #f2f2f2;">
            <th>Indicador</th>
            <th>Valor Ano</th>
            <th>Meta Ano</th>
            <th>Cenário Ano</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: center">{faturamento_ano}</td>
            <td style="text-align: center">{meta_faturamento_ano}</td>
            <td style="text-align: center"><span style="font-size: 18px;">{'🟢' if cor_fat_ano == 'green' else '🔴'}</span></td>
        </tr>
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{qtde_produtos_ano}</td>
            <td style="text-align: center">{meta_qtdprodutos_ano}</td>
            <td style="text-align: center"><span style="font-size: 18px;">{'🟢' if cor_qtde_ano == 'green' else '🔴'}</span></td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">{ticket_medio_ano:.2f}</td>
            <td style="text-align: center">{meta_ticketmedio_ano}</td>
            <td style="text-align: center"><span style="font-size: 18px;">{'🟢' if cor_ticket_ano == 'green' else '🔴'}</span></td>
        </tr>
        </table>

        <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>
        <p>Qualquer dúvida estou à disposição.</p>
        <p>Att.,<br>Lira</p>
    </body>
    </html>
    ''', subtype='html')



    # Cria o caminho completo para o arquivo da loja, com base na data do dia_indicador
    attachment = (
        pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'  # Usa Path.cwd() para garantir que parte do caminho se refere à pasta onde o script está sendo executado

    )
    #Garante que o arquivo será automaticamente fechado após o uso (mesmo se der erro).
    #Abre o arquivo Excel em modo binário para leitura, e garante que ele será fechado corretamente depois que você terminar de usá-lo.
    with open(attachment, 'rb') as f: 
        file_data = f.read()  ## Lê todo o conteúdo binário
        file_name = attachment.name ## Pega apenas o nome do arquivo (ex: "Norte Shopping.xlsx")
        msg.add_attachment(
            file_data,
            maintype='application',
            subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet',
            filename=file_name
        )

    #Abre uma conexão segura com o servidor do Gmail usando SMTP_SSL na porta 465
    #Faz login com o e-mail e a senha de app
    #Envia a mensagem com todos os dados (destinatário, corpo e anexo)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(usuario, senha)
        smtp.send_message(msg)
    print('E-mail da loja {} enviado'.format(loja))


E-mail da loja Iguatemi Esplanada enviado
E-mail da loja Shopping Midway Mall enviado
E-mail da loja Norte Shopping enviado
E-mail da loja Shopping Iguatemi Fortaleza enviado
E-mail da loja Shopping União de Osasco enviado
E-mail da loja Shopping Center Interlagos enviado
E-mail da loja Rio Mar Recife enviado
E-mail da loja Salvador Shopping enviado
E-mail da loja Rio Mar Shopping Fortaleza enviado
E-mail da loja Shopping Center Leste Aricanduva enviado
E-mail da loja Ribeirão Shopping enviado
E-mail da loja Shopping Morumbi enviado
E-mail da loja Parque Dom Pedro Shopping enviado
E-mail da loja Bourbon Shopping SP enviado
E-mail da loja Palladium Shopping Curitiba enviado
E-mail da loja Passei das Águas Shopping enviado
E-mail da loja Center Shopping Uberlândia enviado
E-mail da loja Shopping Recife enviado
E-mail da loja Shopping Vila Velha enviado
E-mail da loja Shopping SP Market enviado
E-mail da loja Shopping Eldorado enviado
E-mail da loja Shopping Ibirapuera enviado
E-mail da l

### Criar Ranking Diretoria

In [12]:
faturamento_lojas_anual = vendas.groupby('Loja')[['Valor Final']].sum().sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_anual)

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_anual.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))


vendas_dia = vendas.loc[vendas['Data'] == dia_indicador,:]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Valor Final']].sum().sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_dia)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

,Valor Final
Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


,Valor Final
Loja,
Salvador Shopping,3950
Novo Shopping Ribeirão Preto,3400
Center Shopping Uberlândia,2651
Shopping Eldorado,2391
Shopping Center Interlagos,1582
Shopping Recife,1366
Norte Shopping,1259
Shopping União de Osasco,1207
Shopping Vila Velha,937


### Enviar o E-mail para a Diretoria

In [24]:
load_dotenv()

# Recupera as credenciais de forma segura
usuario = os.getenv('usuario')
senha = os.getenv('senha')
destinatario = email.loc[email['Loja'] == 'Diretoria', 'E-mail'].values[0]

msg = EmailMessage()
msg['From'] = usuario
msg['To'] = destinatario
msg['Subject'] = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'
msg.set_content(f'''
Prezados, bom dia.

Segue o resumo dos destaques em faturamento:

🔹 Melhor loja do dia: Loja {faturamento_lojas_dia.index[0]} com faturamento de R$ {faturamento_lojas_dia.iloc[0, 0]:.2f}
🔹 Pior loja do dia: Loja {faturamento_lojas_dia.index[-1]} com faturamento de R$ {faturamento_lojas_dia.iloc[-1, 0]:.2f}

🔹 Melhor loja do ano: Loja {faturamento_lojas_anual.index[0]} com faturamento acumulado de R$ {faturamento_lojas_anual.iloc[0,0]:.2f}
🔹 Pior loja do ano: Loja {faturamento_lojas_anual.index[-1]} com faturamento acumulado de R$ {faturamento_lojas_anual.iloc[-1, 0]:.2f}

Em anexo, envio os rankings de faturamento diário e anual de todas as lojas.

Qualquer dúvida, fico à disposição.

Atenciosamente,
Lira

''' ) # Corpo do e-mail (modo texto)

# Cria o caminho completo para o arquivo da loja, com base na data do dia_indicador
attachment_dia = (
    pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
)

attachment_ano = (
    pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Dia.xlsx'
)

# Anexa o Ranking Anual
with open(attachment_ano, 'rb') as f: 
    file_data = f.read()
    file_name = attachment_ano.name
    msg.add_attachment(
        file_data,
        maintype='application',
        subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet',
        filename=file_name
    )
# Anexa o Ranking Diário
with open(attachment_dia, 'rb') as f: 
    file_data = f.read()
    file_name = attachment_dia.name
    msg.add_attachment(
        file_data,
        maintype='application',
        subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet',
        filename=file_name
    )

# Abre uma conexão segura com o servidor do Gmail usando SMTP_SSL na porta 465
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
    smtp.login(usuario, senha)
    smtp.send_message(msg)

print('E-mail da Diretoria enviado')
# Envia o e-mail para a diretoria com os rankings do dia e do ano


E-mail da Diretoria enviado
